<a href="https://colab.research.google.com/github/uc2045/genai-projects/blob/master/030925_Build_a_Multi_User_Conversational_Tool_Calling_Agentic_AI_Research_Assistant_March2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Build a Multi-User Conversational Tool-Calling Agentic AI Research Assistant with LangChain

This demo will cover building AI Agents with the legacy LangChain `AgentExecutor`. These are fine for getting started, but for working with more advanced agents, LangChain recommends to use LangGraph, which we will cover in the future demos.

Agents are systems that use an LLM as a reasoning engine to determine which actions to take and what the inputs to those actions should be. The results of those actions can then be fed back into the agent and it determines whether more actions are needed, or whether it is okay to stop.

Here we will build a multi-user Conversational Agent which can refer to previous conversations and give more contextual answers by storing agent messages to a SQL database

![](https://i.imgur.com/lHWqaT9.png)



## Install OpenAI, and LangChain dependencies

In [ ]:
!pip install langchain==0.3.19
!pip install langchain-openai==0.3.8
!pip install langchain-community==0.3.19

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 415.4/415.4 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 21.8 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.40
    Uninstalling langchain-core-0.3.40:
      Successfully uninstalled langchain-core-0.3.40
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 42.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 47.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.6 MB/s eta 0:00:00
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.19
    Uninstalling langchain-0.3.19:
      Successfully uninstalled langchain-0.3.19


## Enter Open AI API Key

In [ ]:
from getpass import getpass

OPENAI_KEY = getpass('Enter Open AI API Key: ')

Enter Open AI API Key: ··········


## Enter Tavily Search API Key

Get a free API key from [here](https://tavily.com/#api)

In [ ]:
TAVILY_API_KEY = getpass('Enter Tavily Search API Key: ')

Enter Tavily Search API Key: ··········


## Enter WeatherAPI API Key

Get a free API key from [here](https://www.weatherapi.com/signup.aspx)

In [ ]:
WEATHER_API_KEY = getpass('Enter WeatherAPI API Key: ')

Enter WeatherAPI API Key: ··········


## Setup Environment Variables

In [ ]:
import os

os.environ['OPENAI_API_KEY'] = OPENAI_KEY
os.environ['TAVILY_API_KEY'] = TAVILY_API_KEY

## Create Tools

Here we create two custom tools which are wrappers on top of the [Tavily API](https://tavily.com/#api) and [WeatherAPI](https://www.weatherapi.com/)

- Web Search tool with information extraction
- Weather tool

![](https://i.imgur.com/EnrAbJh.png)

In [ ]:
from langchain_community.utilities.tavily_search import TavilySearchAPIWrapper
from langchain_core.tools import tool
import requests

tavily_search = TavilySearchAPIWrapper()

@tool
def search_web_extract_info(query: str) -> list:
    """Search the web for a query. Userful for general information or general news"""
    results = tavily_search.raw_results(query=query,
                                        max_results=8,
                                        search_depth='advanced',
                                        include_answer=False,
                                        include_raw_content=True) # it will also scrape the web pages
    docs = results['results']
    docs = ['## Title'+'\n\n'+doc['title']+'\n\n'+'## Content'+'\n\n'+doc['raw_content'] for doc in docs]
    return docs


@tool
def get_weather(query: str) -> list:
    """Search weatherapi to get the current weather."""
    base_url = "http://api.weatherapi.com/v1/current.json"
    complete_url = f"{base_url}?key={WEATHER_API_KEY}&q={query}"

    response = requests.get(complete_url)
    data = response.json()
    if data.get("location"):
        return data
    else:
        return "Weather Data Not Found"

## Build and Test AI Agent

Now that we have defined the tools and the LLM, we can create the agent. We will be using a tool calling agent to bind the tools to the agent with a prompt. We will also add in the capability to store historical conversations as memory

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

SYS_PROMPT = """Act as a helpful assistant.
                You run in a loop of Thought, Action, PAUSE, Observation.
                At the end of the loop, you output an Answer.
                Use Thought to describe your thoughts about the question you have been asked.
                Use Action to run one of the actions available to you - then return PAUSE.
                Observation will be the result of running those actions.
                Repeat till you get to the answer for the given user query.

                Use the following workflow format:
                  Question: the input task you must solve
                  Thought: you should always think about what to do
                  Action: the action to take which can be any of the following:
                            - break it into smaller steps if needed
                            - see if you can answer the given task with your trained knowledge
                            - call the most relevant tools at your disposal mentioned below in case you need more information
                  Action Input: the input to the action
                  Observation: the result of the action
                  ... (this Thought/Action/Action Input/Observation can repeat N times)
                  Thought: I now know the final answer
                  Final Answer: the final answer to the original input question

                Tools at your disposal to perform tasks as needed:
                  - get_weather: whenever user asks get the weather of a place.
                  - search_web_extract_info: whenever user asks for specific information or if you don't know the answer.
             """

prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", SYS_PROMPT),
        MessagesPlaceholder(variable_name="history", optional=True),
        ("human", "{query}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

prompt_template.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template="Act as a helpful assistant.\n                You run in a loop of Thought, Action, PAUSE, Observation.\n                At the end of the loop, you output an Answer.\n                Use Thought to describe your thoughts about the question you have been asked.\n                Use Action to run one of the actions available to you - then return PAUSE.\n                Observation will be the result of running those actions.\n                Repeat till you get to the answer for the given user query.\n\n                Use the following workflow format:\n                  Question: the input task you must solve\n                  Thought: you should always think about what to do\n                  Action: the action to take which can be any of the following:\n                            - break it into smaller steps if needed\n                            - see if you can

Now, we can initalize the agent with the LLM, the prompt, and the tools.

The agent is responsible for taking in input and deciding what actions to take.

REMEMBER the Agent does not execute those actions - that is done by the AgentExecutor

Note that we are passing in the model `chatgpt`, not `chatgpt_with_tools`.

That is because `create_tool_calling_agent` will call `.bind_tools` for us under the hood.

This should ideally be used with an LLM which supports tool \ function calling

In [ ]:
from langchain.agents import create_tool_calling_agent
from langchain_openai import ChatOpenAI

chatgpt = ChatOpenAI(model="gpt-4o", temperature=0)
tools = [search_web_extract_info, get_weather]
agent = create_tool_calling_agent(chatgpt, tools, prompt_template)

Finally, we combine the `agent` (the brains) with the `tools` inside the `AgentExecutor` (which will repeatedly call the agent and execute tools).

In [ ]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent,
                               tools=tools,
                               early_stopping_method='force',
                               max_iterations=10)

In [ ]:
query = """Summarize the key points discussed in Nvidia's Q4 2024 earnings call"""
response = chatgpt.invoke(query)
print(response.content)

As of my last update, Nvidia's Q4 2024 earnings call has not occurred. However, I can provide a general idea of what typically happens during such calls. In an earnings call, Nvidia's executives would likely discuss:

1. **Financial Performance**: Key financial metrics such as revenue, net income, earnings per share, and operating margins. They might compare these figures to previous quarters and the same quarter in the prior year.

2. **Segment Performance**: Insights into how different business segments performed, such as gaming, data center, professional visualization, and automotive.

3. **Market Trends**: Discussion on trends affecting Nvidia's business, such as demand for GPUs, developments in AI and machine learning, and the impact of cryptocurrency mining on GPU sales.

4. **Product Updates**: Information on new product launches, updates to existing products, and future product roadmaps.

5. **Strategic Initiatives**: Updates on strategic partnerships, acquisitions, or investme

In [ ]:
query = """Summarize the key points discussed in Nvidia's Q4 2024 earnings call"""
response = agent_executor.invoke({"query": query})

In [ ]:
response

{'query': "Summarize the key points discussed in Nvidia's Q4 2024 earnings call",
 'output': "Nvidia's Q4 2024 earnings call highlighted several key points:\n\n1. **Financial Performance**: Nvidia reported a record quarterly revenue of $22.1 billion, marking a 22% increase from the previous quarter and a 265% rise from the same period a year ago. The earnings per share (EPS) also saw significant growth, with GAAP EPS at $4.93 and Non-GAAP EPS at $5.16.\n\n2. **Data Center Growth**: The data center segment was a major contributor, with revenue reaching $18.4 billion, up 27% sequentially and 409% year-over-year. This growth was driven by the demand for AI inference and the transition to accelerated computing.\n\n3. **AI and Accelerated Computing**: Nvidia emphasized the growing demand for AI and accelerated computing, which are becoming central to their strategy. The company is focusing on AI inference, which contributed significantly to the data center revenue.\n\n4. **Product and Marke

In [ ]:
from IPython.display import display, Markdown

display(Markdown(response['output']))

Nvidia's Q4 2024 earnings call highlighted several key points:

1. **Financial Performance**: Nvidia reported a record quarterly revenue of $22.1 billion, marking a 22% increase from the previous quarter and a 265% rise from the same period a year ago. The earnings per share (EPS) also saw significant growth, with GAAP EPS at $4.93 and Non-GAAP EPS at $5.16.

2. **Data Center Growth**: The data center segment was a major contributor, with revenue reaching $18.4 billion, up 27% sequentially and 409% year-over-year. This growth was driven by the demand for AI inference and the transition to accelerated computing.

3. **AI and Accelerated Computing**: Nvidia emphasized the growing demand for AI and accelerated computing, which are becoming central to their strategy. The company is focusing on AI inference, which contributed significantly to the data center revenue.

4. **Product and Market Expansion**: Nvidia is expanding its product offerings and market reach, with strategic initiatives in AI and partnerships with major cloud service providers. Despite regulatory challenges in China, Nvidia is adapting by offering alternative products.

5. **Future Outlook**: Nvidia provided strong guidance for the next quarter, expecting continued growth driven by AI and accelerated computing. The company is optimistic about its future prospects, supported by its strategic focus and partnerships.

Overall, Nvidia's Q4 2024 earnings call underscored its strong financial performance, strategic focus on AI and accelerated computing, and positive outlook for future growth.

In [ ]:
query = """Summarize the key points discussed in Intel's Q4 2024 earnings call"""
response = agent_executor.invoke({"query": query})
display(Markdown(response['output']))

Intel's Q4 2024 earnings call highlighted several key points:

1. **Financial Performance**: Intel reported Q4 revenue of $14.3 billion, surpassing guidance and showing a 7% sequential increase. The non-GAAP gross margin was 42.1%, and earnings per share (EPS) were $0.13, slightly above the guidance of $0.12. However, the full-year 2024 revenue was $53.1 billion, a 2.1% decline year-over-year, with a full-year EPS of minus $0.13.

2. **Segment Performance**: The Client Computing Group (CCG) generated $8.02 billion in revenue, down 9% year-over-year. The Data Center and AI (DCAI) segment reported $3.39 billion in revenue, a 3% decline. The Network and Edge (NEX) unit saw a 10% increase in revenue to $1.62 billion.

3. **Strategic Initiatives**: Intel is focusing on strengthening its product portfolio and process roadmap. The company plans to launch Panther Lake, its lead product on the Intel 18A process technology, in the second half of 2025. Intel is also working on Jaguar Shores to address the AI data center market more broadly.

4. **Challenges and Outlook**: Intel faces increased competition, particularly in the AI PC category, and is not yet significantly participating in the cloud-based AI data center market. The company expects Q1 2025 revenue to be between $11.7 billion and $12.7 billion, with a breakeven EPS on a non-GAAP basis.

5. **Operational Changes**: Intel is implementing a cost reduction plan and restructuring its business to improve efficiency and profitability. The company is also exploring the establishment of Intel Foundry as an independent subsidiary to enhance governance and operational separation.

6. **Leadership Transition**: Intel appointed two interim co-CEOs, Michelle Johnston Holthaus and David Zinsner, following the departure of former CEO Pat Gelsinger. The search for a permanent CEO is ongoing.

7. **Government Support**: Intel finalized a $7.86 billion U.S. government grant to support manufacturing, reflecting its strategic importance in domestic semiconductor production.

Overall, Intel is navigating market challenges while focusing on innovation and operational efficiency to enhance its competitive position and create shareholder value.

In [ ]:
query = """which company's future outlook looks to be better?
        """
response = agent_executor.invoke({"query": query})
display(Markdown(response['output']))

To provide an accurate assessment of a company's future outlook, I would need to know which specific companies you are interested in comparing. Please provide the names of the companies you would like to evaluate.

In [ ]:
query = """Tell me about the weather in Zurich, show detailed statistics
        """
response = agent_executor.invoke({"query": query})
display(Markdown(response['output']))

The current weather in Zurich, Switzerland is as follows:

- **Temperature**: 16.4°C (61.5°F)
- **Condition**: Sunny
- **Wind**: 4.3 mph (6.8 kph) from the NE
- **Pressure**: 1010.0 mb (29.83 in)
- **Precipitation**: 0.0 mm (0.0 in)
- **Humidity**: 27%
- **Cloud Cover**: 0%
- **Feels Like**: 16.4°C (61.5°F)
- **Wind Chill**: 12.8°C (55.1°F)
- **Heat Index**: 13.1°C (55.5°F)
- **Dew Point**: 3.1°C (37.6°F)
- **Visibility**: 10.0 km (6.0 miles)
- **UV Index**: 0.3
- **Wind Gusts**: 6.4 mph (10.3 kph)

The weather is currently sunny with no cloud cover, and the conditions are quite pleasant with a light breeze.

The agent is doing pretty well but unfortunately it doesn't remember conversations. We will use some user-session based memory to store this and dive deeper into this in the next video.

## Build and Test Multi-User Conversational AI Agent

We will now use `SQLChatMessageHistory` which we learnt in the previous module to store separate conversation histories per user or session.

This will help us build a conversational Agentic Chatbot which will be accessed by many users at the same time

In [ ]:
# removes the memory database file - usually not needed
# you can run this only when you want to remove ALL conversation histories
# ok if you get rm: cannot remove 'memory.db': No such file or directory  because initially no memory exists
!rm memory.db

rm: cannot remove 'memory.db': No such file or directory


In [ ]:
from langchain_community.chat_message_histories import SQLChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

# used to retrieve conversation history from database
# based on a specific user or session ID
def get_session_history_db(session_id):
    return SQLChatMessageHistory(session_id, "sqlite:///memory.db")

# create a conversation chain + agent which can load memory based on specific user or session id
agentic_chatbot = RunnableWithMessageHistory(
    agent_executor,
    get_session_history_db,
    input_messages_key="query",
    history_messages_key="history",
)

# function to call the agent show results per user session
from IPython.display import display, Markdown
def chat_with_agent(prompt: str, session_id: str):
    response = agentic_chatbot.invoke({"query": prompt},
                                      {'configurable': { 'session_id': session_id}})
    display(Markdown(response['output']))

Let's now simulate User 1 using the agent

In [ ]:
user_id = 'john001'
prompt = "Summarize the key points discussed in Nvidia's Q4 2024 earnings call"
chat_with_agent(prompt, user_id)

/usr/local/lib/python3.11/dist-packages/langchain_core/runnables/history.py:606: LangChainDeprecationWarning: `connection_string` was deprecated in LangChain 0.2.2 and will be removed in 1.0. Use connection instead.
  message_history = self.get_session_history(


Nvidia's Q4 2024 earnings call highlighted several key points:

1. **Financial Performance**: Nvidia reported a record quarterly revenue of $22.1 billion, marking a 22% increase from the previous quarter and a 265% rise from the same period a year ago. The earnings per share (EPS) also saw significant growth, with GAAP EPS at $4.93 and Non-GAAP EPS at $5.16.

2. **Data Center Growth**: The data center segment was a major contributor, with revenue reaching $18.4 billion, up 27% sequentially and 409% year-over-year. This growth was driven by the demand for AI inference and the transition to accelerated computing.

3. **AI and Accelerated Computing**: Nvidia emphasized the surging demand for accelerated computing and generative AI across various industries. The company is focusing on AI inference, which contributed to about 40% of the data center's revenue.

4. **Strategic Initiatives**: Nvidia is expanding its strategic collaborations, including partnerships with Google and Amazon Web Services, to enhance its AI platforms. The company is also addressing regulatory challenges in China by offering alternative products.

5. **Future Outlook**: Nvidia provided a strong outlook for the next quarter, expecting revenue to be around $24 billion. The company is optimistic about its growth trajectory, driven by its focus on AI and accelerated computing.

6. **Market Sentiment**: Following the earnings announcement, Nvidia's stock saw a positive reaction, reflecting investor confidence in the company's strategic direction and growth potential.

Overall, Nvidia's Q4 2024 earnings call underscored its strong financial performance, strategic focus on AI and accelerated computing, and positive market sentiment.

In [ ]:
prompt = "What about Intel?"
chat_with_agent(prompt, user_id)

Intel's Q4 2024 earnings call highlighted several key points:

1. **Financial Performance**: Intel reported Q4 revenue of $14.3 billion, which was a 7% sequential increase but a 7% year-over-year decline. The earnings per share (EPS) was $0.13, slightly above the guidance of $0.12. The full-year 2024 revenue was $53.1 billion, down 2.1% from the previous year.

2. **Segment Performance**: The Client Computing Group (CCG) generated $8.02 billion in revenue, down 9% year-over-year. The Data Center and AI segment reported $3.39 billion in revenue, a 3% decline. The Network and Edge unit saw a 10% increase in revenue to $1.62 billion.

3. **Challenges and Strategy**: Intel is facing increased competition, particularly in the AI data center market, and is not yet participating meaningfully in the cloud-based AI data center market. The company is focusing on product innovation and partnerships to improve its market position.

4. **Intel Foundry Services**: Intel Foundry reported $4.5 billion in revenue, with a significant operating loss. The company aims to achieve breakeven operating income for Intel Foundry by the end of 2027.

5. **Future Outlook**: Intel provided a Q1 2025 revenue guidance of $11.7 billion to $12.7 billion, with a breakeven EPS. The company expects gross margins to improve throughout the year, driven by a mix shift to EUV wafers and growth in advanced packaging.

6. **Strategic Initiatives**: Intel is focusing on its 18A process technology, with plans for volume chip production in the second half of 2025. The company is also working on restructuring its business to improve efficiency and profitability.

Overall, Intel's Q4 2024 earnings call reflected a challenging environment with a focus on strategic initiatives to improve competitiveness and financial performance.

In [ ]:
prompt = "Which company seems to be doing better?"
chat_with_agent(prompt, user_id)

Based on the summaries of the Q4 2024 earnings calls for both Nvidia and Intel, Nvidia appears to be performing better than Intel. Here are some key points that highlight Nvidia's stronger position:

1. **Revenue Growth**: Nvidia reported a record quarterly revenue of $22.1 billion, with a significant year-over-year increase of 265%. In contrast, Intel's revenue for the same period was $14.3 billion, which represented a year-over-year decline of 7%.

2. **Profitability**: Nvidia's earnings per share (EPS) showed substantial growth, with both GAAP and Non-GAAP EPS significantly higher than the previous year. Intel's EPS was slightly above guidance but did not show the same level of growth as Nvidia.

3. **Market Demand**: Nvidia is benefiting from strong demand in the AI and accelerated computing sectors, particularly in data centers. This demand is driving significant growth in their data center segment. Intel, on the other hand, is facing challenges in the AI data center market and is not yet participating meaningfully in this area.

4. **Future Outlook**: Nvidia provided a strong revenue outlook for the next quarter, expecting around $24 billion, indicating continued growth momentum. Intel's guidance for the next quarter shows a more modest revenue range and a breakeven EPS, suggesting a more cautious outlook.

5. **Market Sentiment**: Nvidia's stock saw a positive reaction following the earnings announcement, reflecting investor confidence in its strategic direction. Intel's performance and outlook appear to be more conservative, with ongoing challenges in certain segments.

Overall, Nvidia's strong financial performance, growth in key market segments, and positive future outlook suggest that it is currently doing better than Intel.

Let's now simulate User 2 using the agent

In [ ]:
user_id = 'bond007'
prompt = "how is the weather in Bangalore today? Show detailed statistics"
chat_with_agent(prompt, user_id)

The weather in Bangalore today is sunny with the following detailed statistics:

- **Temperature**: 33.2°C (91.8°F)
- **Feels Like**: 31.1°C (87.9°F)
- **Wind**: 8.7 mph (14.0 kph) from the NE
- **Wind Gusts**: Up to 10.0 mph (16.2 kph)
- **Humidity**: 20%
- **Pressure**: 1011.0 mb (29.85 in)
- **Precipitation**: 0.0 mm (0.0 in)
- **Cloud Cover**: 0% (Clear skies)
- **Visibility**: 6.0 km (3.0 miles)
- **UV Index**: 1.7
- **Dew Point**: -2.3°C (27.9°F)

The weather is clear and sunny, making it a good day for outdoor activities.

In [ ]:
user_id = 'bond007'
prompt = "what about Dubai?"
chat_with_agent(prompt, user_id)

The weather in Dubai today is sunny with the following detailed statistics:

- **Temperature**: 37.1°C (98.8°F)
- **Feels Like**: 40.1°C (104.2°F)
- **Wind**: 7.4 mph (11.9 kph) from the SW
- **Wind Gusts**: Up to 15.5 mph (24.9 kph)
- **Humidity**: 9%
- **Pressure**: 1008.0 mb (29.77 in)
- **Precipitation**: 0.0 mm (0.0 in)
- **Cloud Cover**: 0% (Clear skies)
- **Visibility**: 10.0 km (6.0 miles)
- **UV Index**: 6.4
- **Dew Point**: 12.4°C (54.4°F)

The weather is hot and sunny, typical for Dubai, so it's advisable to stay hydrated and use sun protection if you're outdoors.

In [ ]:
user_id = 'bond007'
prompt = "which city is hotter?"
chat_with_agent(prompt, user_id)

Dubai is hotter than Bangalore today. The temperature in Dubai is 37.1°C (98.8°F), while in Bangalore, it is 33.2°C (91.8°F).